In [5]:
import pandas as pd
data = pd.read_csv('BANKNIFTY.csv')

In [6]:
# stock_data=data

In [7]:
def open_range_breakout(daily_data):
    
    
    open_range_high = daily_data[daily_data['Time'].dt.strftime('%H:%M').between('09:15', '09:30')]['High'].max()
    open_range_low = daily_data[daily_data['Time'].dt.strftime('%H:%M').between('09:15', '09:30')]['Low'].min()

    position = None
    entry_price = None
    exit_price = None
    trade_taken = False

    for index, row in daily_data.iterrows():
        if not trade_taken and pd.to_datetime(row['Time']).strftime('%H:%M') >= '09:30':
            if row['High'] > open_range_high:
                position = 'buy'
                entry_price = row['High']
                stop_loss = entry_price * 0.995
                trade_taken = True
                decision_data = pd.read_csv('Decision.csv')
                decision_data = pd.DataFrame([{
                        'Date': daily_data.iloc[0]['Date'],
                        'Time': pd.to_datetime(row['Time']).strftime('%H:%M'),
                        'Position': position,
                        'Entry Price': entry_price,
                    }])
                decision_data.to_csv('Decision.csv', mode='a', header=False, index=False)
            elif row['Low'] < open_range_low:
                position = 'sell'
                entry_price = row['Low']
                stop_loss = entry_price * 1.005
                trade_taken = True
                decision_data = pd.read_csv('Decision.csv')
                decision_data = pd.DataFrame([{
                        'Date': daily_data.iloc[0]['Date'],
                        'Time': pd.to_datetime(row['Time']).strftime('%H:%M'),
                        'Position': position,
                        'Entry Price': entry_price,
                    }])
                decision_data.to_csv('Decision.csv', mode='a', header=False, index=False)

        if position is not None:
            if (position == 'buy' and row['Open'] <= stop_loss) or (position == 'sell' and row['Open'] >= stop_loss):
                exit_price = row['Open']
                decision_data = pd.read_csv('Decision.csv')
                decision_data = pd.DataFrame([{
                        'Date': daily_data.iloc[0]['Date'],
                        'Time': pd.to_datetime(row['Time']).strftime('%H:%M'),
                        'Position': position+' Square Off',
                        'Entry Price' : "",
                        'Exit Price': exit_price,
                    }])
                decision_data.to_csv('Decision.csv', mode='a', header=False, index=False)
                 
                break
            elif pd.to_datetime(row['Time']).strftime('%H:%M') == '15:15':
                exit_price = row['Close']
                decision_data = pd.read_csv('Decision.csv')
                decision_data = pd.DataFrame([{
                        'Date': daily_data.iloc[0]['Date'],
                        'Time': pd.to_datetime(row['Time']).strftime('%H:%M'),
                        'Position': position+' Square Off',
                        'Entry Price' : "",
                        'Exit Price': exit_price,
                    }])
                decision_data.to_csv('Decision.csv', mode='a', header=False, index=False)

    if position is not None:
        profit = 0
        loss = 0

        if position == 'buy' and exit_price >= entry_price:
            profit = exit_price - entry_price
        elif position == 'buy' and exit_price < entry_price:
            loss = exit_price - entry_price
        elif position == 'sell' and exit_price > entry_price :
            loss = entry_price - exit_price
        elif position =='sell' and exit_price <= entry_price : 
            profit = entry_price - exit_price

        total_profit_loss = profit + loss

        # print("Date:", daily_data.iloc[0]['Date'])
        # print("Position:", position)
        # print("Entry Price:", entry_price)
        # print("Exit Price:", exit_price)
        # print("Profit:", profit)
        # print("Loss:", loss)
        # print("Total Profit/Loss:", total_profit_loss)
        # print("--------------------------------------")
        
        profit_loss_record = pd.read_csv('ProfitLoss2020.csv')
        # Append records to profit_loss_data DataFrame
        profit_loss_record = pd.DataFrame([{
            'Date': daily_data.iloc[0]['Date'],
            'Position': position,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Profit': profit,
            'Loss': loss,
            'Total Profit/Loss': total_profit_loss
        }])
        profit_loss_record.to_csv('ProfitLoss2020.csv', mode='a', header=False, index=False)


In [8]:
# Load the BankNifty data
data = pd.read_csv('BANKNIFTY.csv')

# Convert the Time column to datetime dtype
data['Time'] = pd.to_datetime(data['Time'])

# Create decision CSV file
decision_data = pd.DataFrame(columns=['Date','Time','Position','Entry Price','Exit Price'])
profit_loss_data = pd.DataFrame(columns=['Date','Position','Entry Price','Exit Price','Profit','Loss','Total Profit/Loss'])
decision_data.to_csv('Decision.csv',index=False)
profit_loss_data.to_csv('ProfitLoss2020.csv',index=False)

# Apply the open_range_breakout function to each day's data
data.groupby('Date').apply(open_range_breakout)

""
